In [ ]:
from tensorflow.keras.datasets import mnist #케라스(tensorflow >2.0)에서 받을수 있는 데이터 셋
#from tensorflow.keras.utils import np_utils
from tensorflow.python.keras.utils.np_utils import to_categorical #np_utils 모듈 수정됨, 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D #Dropout, Flatten, Conv2D 등 import
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping #모델저장, 과적합 방지

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [ ]:
# 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() #데이터 불러오기
#총7만개 자료, 6만개 Train데이터 1만개 Test데이터
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
#28, 28 , 1 --> 28*28픽셀 흑백영상이라 채널이 1 
#0 ~ 255로 분포되어 있는 픽셀값을 0~1사이의 실수값으로 변경하기 위해 255로 나눔
#파이썬에서 / 는 소수점까지 구해진다는 것을 잊으면 안됨 
#정수로 나누고 싶다면 //
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
Y_train = to_categorical(Y_train) #손글씨 결과를 0~9까지 표현하는데 원-핫인코딩 결과로 리턴
Y_test = to_categorical(Y_test) #테스트 결과를 원-핫인코딩함, 이유는 텐서플로우에서 효율적인 계산을 위해서

In [ ]:
# 컨볼루션 신경망의 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

#CNN 설명을 기억하시면 됩니다.
#Conv2D 32의 Mask출력을 하고 Mask의 크기가 3*3, 입력값은 손글씨 영상
#MaxPooling2D 컨볼루션을 통한 이미지 특징검출 결과 크고 복잡하다고 생각되면
#데이터양의 축소가 필요함 pool_size = 2는 크기를 절반으로 만듦
#Dropout은 딥러닝 학습중 25% 확률로 노드를 꺼버림. 
#속도계선 및 과적합 방지의 방법으로 사용됨
#Flatten 이미지는 2차원 배열인데 활성함수는 1차원 함수들이 통과되므로 활성함수 통과 전
#1차원 함수로 만들어서 연산을 수행함

#softmax는 분류 - 지금은 손글씨 10개 따라서 출력이 10개

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# 모델의 실행
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10,
                    batch_size=500, verbose=1,
                    callbacks=[early_stopping_callback,checkpointer])

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

In [ ]:
# 테스트 셋의 오차
y_vloss = history.history['val_loss']
# 학습셋의 오차
y_loss = history.history['loss']

In [ ]:
# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.figure(figsize=(12,12))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')
# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
predict = model.predict(X_test)
predict

In [ ]:
predict01 = np.argmax(predict, 1)
predict01

In [ ]:
fig = plt.figure()
for i in range(20):
    subplot = fig.add_subplot(4, 5, i+1)
    
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title('predict: %d' %predict01[i])
    
    subplot.imshow(X_test[i].reshape((28, 28)), cmap=plt.cm.gray_r)
    
plt.show()